# **Import Dataset**

In [ ]:
!pip3 install pickle5
import pickle5 as pc


import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
from datetime import datetime
import tensorflow as tf

# set random seed for random weights
np.random.seed(1)
tf.random.set_seed(1)
data_train = pd.read_pickle('data_train') #full set
data_train = data_train[data_train.ID <= 100] # Change this for 100, 500 or 1000 training structures
data_val = pd.read_pickle('data_val')
data_test = pd.read_pickle('data_test')

fig, ax = plt.subplots(figsize=(7,5))

ax.plot(data_train[data_train.ID == 1]['cycle'],data_train[data_train.ID == 1]['gauge1'],label= 'gauge1')
ax.plot(data_train[data_train.ID == 1]['cycle'],data_train[data_train.ID == 1][ 'gauge2'],label= 'gauge2')
ax.plot(data_train[data_train.ID == 1]['cycle'],data_train[data_train.ID == 1]['gauge3'], label= 'gauge3')
ax.grid()
ax.legend()

ax.set(xlabel='Cycle', ylabel='Strain')

plt.savefig('training_data_ID_1.pdf', bbox_inches='tight')

plt.show()

plt.figure(figsize=(20, 12))
plt.plot(data_test[data_test.ID == 1]['gauge1'],label= 'gauge1')
plt.plot(data_test[data_test.ID == 1][ 'gauge2'],label= 'gauge2')
plt.plot(data_test[data_test.ID == 1]['gauge3'], label= 'gauge3')
plt.grid()
plt.legend()
plt.show()

# **Categorize Data**

In [ ]:
print(data_train.RUL.describe())
print(data_test.RUL.describe())

l=0.5
nb_bins = 20
lower_bin_bound = 0
upper_bin_bound = 80000

bins = np.linspace(lower_bin_bound, upper_bin_bound**(1-l), nb_bins)**(1/l)
labels=[i for i in range(bins.shape[0]-1)]
plt.plot(bins,np.full(bins.shape[0],0),'o')
print(bins)
print(labels)

data_train['RUL_bins'] = pd.cut(data_train['RUL'], bins=bins, labels=labels)
data_test['RUL_bins'] = pd.cut(data_test['RUL'], bins=bins, labels=labels)

display(data_train)
display(data_test)

# **Build Sequences**

In [ ]:
#utils 
nb_gauges = 3

#prepare forecasting data
def gen_X_sequence(id_df, seq_length, seq_cols,timesteps_pred,  type_data = None):
    ind_start = 0
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0+ind_start, num_elements-seq_length+1-timesteps_pred), range(seq_length+ind_start, num_elements+1-timesteps_pred)):
        yield data_array[start:stop, :]
 

def gen_Y_sequence(id_df, seq_length, seq_cols,timesteps_pred, type_data = None):
    ind_start = 0
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0+ind_start, num_elements-seq_length+1-timesteps_pred), range(seq_length+ind_start, num_elements+1-timesteps_pred)):
        yield data_array[stop-1, :]#data_array[start+1:stop+1, :]
   
#prepare data
seq_cols =  ['gauge'+str(i) for i in range(1,4)]#['label'+str(i) for i in range(1,4)]
seq_cols1 =  ['RUL_bins']
sequence_length = 30
timesteps_pred = 1



#training set
seq_gen = (list(gen_X_sequence(data_tr[data_tr['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                   for id in data_tr['ID'].unique())
# generate sequences and convert to numpy array
dbX = np.concatenate(list(seq_gen))



seq_gen = (list(gen_Y_sequence(data_tr[data_tr['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                   for id in data_tr['ID'].unique())
# generate sequences and convert to numpy array
dbY = np.concatenate(list(seq_gen))#.astype(np.long).reshape(-1,)


print(dbX.shape)
print(dbY.shape)

#validation set
seq_gen = (list(gen_X_sequence(data_val[data_val['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                   for id in data_val['ID'].unique())

# generate sequences and convert to numpy array
dbX_val = np.concatenate(list(seq_gen))#.astype(np.long)

seq_gen = (list(gen_Y_sequence(data_val[data_val['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                   for id in data_val['ID'].unique())
# generate sequences and convert to numpy array
dbY_val = np.concatenate(list(seq_gen))#.astype(np.long).reshape(-1,)


#test set
dbX_test = [data_test[data_test['ID']==id][seq_cols].values[-sequence_length:] for id in data_test['ID'].unique()]
dbX_test = np.asarray(dbX_test)

dbY_test = [data_test[data_test['ID']==id][seq_cols1].values[-1] for id in data_test['ID'].unique()]
dbY_test = np.asarray(dbY_test)



print(dbX_val.shape)
print(dbY_val.shape)

# **Normalize Data and One Hot Encoding**

In [ ]:
from tensorflow.keras.utils import to_categorical

dbY = to_categorical(dbY,nb_bins)
dbY_val = to_categorical(dbY_val,nb_bins)
dbY_test = to_categorical(dbY_test,nb_bins)

print(dbX.shape)
dbX[:,:,0].max()
for k in range(3) :
  a = dbX[:,:,k].max()
  b = dbX[:,:,k].min()
  dbX[:,:,k] = (dbX[:,:,k] - b)/(a-b)
  dbX_val[:,:,k] = (dbX_val[:,:,k] - b)/(a-b)
  dbX_test[:,:,k] = (dbX_test[:,:,k] - b)/(a-b)


dbX
dbX_val


# **Build the model**

In [ ]:
dbY.shape

In [ ]:
from tensorflow import keras
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import *


#model_path = ''
import json
json_file = open(model_path + '/GRU_100_tuned.json')
loaded_model = json_file.read()
json_file.close()
model = model_from_json(loaded_model)
model.load_weights(model_path + '/GRU_100_tuned.h5')
model.get_config()
model.summary()
model.get_layer(index=3).recurrent_dropout

In [ ]:
from tensorflow import keras
from keras.layers.recurrent import LSTM, GRU, RNN
from keras.models import *
from keras.layers import *


history_seq = Input(shape = (dbX.shape[1],dbX.shape[2]))
x = history_seq
x = GRU(32, return_sequences=True, dropout=0, recurrent_dropout=0.1)(x)
x = GRU(32, return_sequences=True, dropout=0, recurrent_dropout=0.1)(x)
x = GRU(32, return_sequences=False, dropout=0, recurrent_dropout=0.1)(x)
x = Activation('relu')(x)
x = Dense(dbY.shape[1], activation = 'softmax')(x)
model = Model(history_seq, x)
model.summary()


In [ ]:
dbX.shape[1]

# **Compile and train**

In [ ]:
dbY.shape

In [ ]:
from tensorflow.keras.optimizers import Adam
import keras.backend as K




List_lr=[0.01, 0.001, 0.0001, 0.00001]
learning_rate = List_lr[0]
lr_step = '0'
i=0
opt = Adam(learning_rate=learning_rate)

List_hist=[]
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
temp_model = model

for i in range (0,len(List_lr)):


  ES = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=2, mode='max')
  mc = keras.callbacks.ModelCheckpoint('best_model_gru_prob.h5', monitor='accuracy', mode='max', save_best_only=True)

  lr_step = str(i)
  batch_size = 4096
  epochs = 500
  K.set_value(temp_model.optimizer.learning_rate, learning_rate)
  history = temp_model.fit(dbX, dbY, batch_size=batch_size, epochs=epochs, validation_data=(dbX_val, dbY_val), callbacks=[mc])
  List_hist.append(history.history['accuracy'])
  List_hist.append(history.history['val_accuracy'])
  List_hist.append(history.history['loss'])
  List_hist.append(history.history['val_loss'])
  lr_step = str(i)
  learning_rate = List_lr[i]
  temp_model=tf.keras.models.load_model('best_model_gru_prob.h5')




  

# **Evaluate**

In [ ]:
from tensorflow.keras.optimizers import Adam
import keras.backend as K

opt = Adam(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.evaluate(dbX_test, dbY_test, batch_size=32)

In [ ]:
model_path = '/content/drive/MyDrive/PIR_perso/models_GRU/100_structures'
# serialize model to JSON
model_json = model.to_json()
with open(model_path + "/GRU_100_tuned.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(model_path + "/GRU_100_tuned.h5")



model.evaluate(dbX_test, dbY_test, batch_size=32)
# summarize history for accuracy
fig, ax = plt.subplots(figsize=(7,5))
acc = List_hist[0] + List_hist[4] + List_hist[8] + List_hist[12]        # concatenate history for the 3 learning rate to make it a single learning
val_acc = List_hist[1] + List_hist[5] + List_hist[9] + List_hist[13]
loss = List_hist[2] + List_hist[6] + List_hist[10] + List_hist[14]
val_loss = List_hist[3] + List_hist[7] + List_hist[11] + List_hist[15]
plt.plot(acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
fig.savefig(model_path + '/GRU_100_tuned_acc.png')
# summarize history for loss
fig, ax = plt.subplots(figsize=(7,5))
plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
fig.savefig(model_path + '/GRU_100_tuned_loss.png')